In [1]:
import lmfit
from lmfit.models import GaussianModel
import numpy as np
spectrum = np.genfromtxt('1.dat')
from matplotlib import pyplot as plt
import rampy as rp
import scipy

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(spectrum[:,0],spectrum[:,1],'k.',markersize=3)
plt.xlabel("Raman shift, cm$^{-1}$", fontsize = 12)
plt.ylabel("Normalized intensity, a. u.", fontsize = 12)
plt.ylim(0,4800)
plt.title("Fig. 1: the raw data",fontsize = 12,fontweight="bold")

In [ ]:
x_new = np.arange(2600., 2800., 0.5)
y_new = rp.resample(spectrum[:,0], spectrum[:,1], x_new)
spectrum_resample = np.vstack((x_new,y_new)).T

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(spectrum_resample[:,0],spectrum_resample[:,1],'k.',markersize=8)
plt.xlabel("Raman shift, cm$^{-1}$", fontsize = 12)
plt.ylabel("Normalized intensity, a. u.", fontsize = 12)
plt.title("Fig. 2: the resampled data",fontsize = 12,fontweight="bold")

In [ ]:
bir = np.array([(2600,2605),(2770,2775)])
y_corr, y_base = rp.baseline(spectrum_resample[:,0],spectrum_resample[:,1],bir,'poly',polynomial_order=3)

In [ ]:
lb = 2620 # The lower boundary of interest
hb = 2775 # The upper boundary of interest
x = spectrum_resample[:,0]
x_fit = x[np.where((x > lb)&(x < hb))]
y_fit = y_corr[np.where((x > lb)&(x < hb))]
ese0 = np.sqrt(abs(y_fit[:,0]))/abs(y_fit[:,0]) # the relative errors after baseline subtraction
y_fit[:,0] = y_fit[:,0]/np.amax(y_fit[:,0])*10 # normalise spectra to maximum intensity, easier to handle
sigma = abs(ese0*y_fit[:,0]) #calculate good ese

In [ ]:
# create a new plot for showing the spectrum
plt.figure()
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
inp = plt.plot(x,spectrum_resample[:,1],'k-',label='Original')
corr = plt.plot(x,y_corr,'b-',label='Corrected') #we use the sample variable because it is not already normalized...
bas = plt.plot(x,y_base,'r-',label='Baseline')
plt.xlim(lb,2775)
plt.ylim(0,800)
plt.xlabel("Raman shift, cm$^{-1}$", fontsize = 14)
plt.ylabel("Normalized intensity, a. u.", fontsize = 14)
plt.legend()
plt.title('A) Baseline removal')

plt.subplot(1,2,2)
plt.plot(x_fit,y_fit,'k.')
plt.xlim(lb,2775)
plt.ylim(0,12)
plt.xlabel("Raman shift, cm$^{-1}$", fontsize = 14)
plt.title('B) signal to fit')
#plt.tight_layout()
plt.suptitle('Figure 3', fontsize = 14,fontweight = 'bold')

In [ ]:
def residual(n,pars,x,data=None,eps=None):
    model_peak=[]
    model=0
    for i in range(n)+1:
        a = 'a'+ str(i)
        f = 'f'+ str(i)
        l = 'l'+ str(i)
        peak = 'peak'+str(i)
        aa = pars[a].value
        ff = pars[f].value
        ll = pars[l].value
        locals()[peak] = rp.lorentzian(x,aa,ff,ll)
        model_peak.append(locals()[peak])
        model += locals()[peak]
    if data is None: # if we don't have data, the function only returns the direct calculation
        return model, model_peak
    if eps is None: # without errors, no ponderation
        return (model - data)
    return (model - data)/eps # with errors, the difference is ponderated

In [ ]:
params = lmfit.Parameters()
params.add_many(('a1',   5,   True,  0,      None,  None),
                ('f1',   2640,   True, 2620,    2700,  None),
                ('l1',   10,   True,  0,      40,  None),
                ('a2',   5,   True,  0,      None,  None),
                ('f2',   2670,  True, 2620,   2700,  None),
                ('l2',   10,   True,  0,   20,  None))
                # ('a3',   5,    True,    0,      None,  None),
                # ('f3',   2735,  True, 2650,   2775,  None),
                # ('l3',   10,   True,  0,   40,  None))

In [ ]:
params['f1'].vary = False
params['f2'].vary = False
# params['f3'].vary = False

In [ ]:
algo = 'nelder'
result = lmfit.minimize(residual, params, method = algo, args=(x_fit, y_fit[:,0])) # fit data with  nelder model from scipy

In [ ]:
params['f1'].vary = True
params['f2'].vary = True
# params['f3'].vary = True


In [ ]:
result2 = lmfit.minimize(residual, params,method = algo, args=(x_fit, y_fit[:,0])) # fit data with leastsq model from scipy

In [ ]:
model = lmfit.fit_report(result2.params)
yout, peak1,peak2 = residual(result2.params,x_fit) # the different peaks
rchi2 = (1/(float(len(y_fit))-15-1))*np.sum((y_fit - yout)**2/sigma**2) # calculation of the reduced chi-square

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(x_fit,y_fit,'k-')
plt.plot(x_fit,yout,'r-')
plt.plot(x_fit,peak1,'b-')
plt.plot(x_fit,peak2,'b-')

plt.xlim(lb,hb)
plt.ylim(-0.5,10.5)
plt.xlabel("Raman shift, cm$^{-1}$", fontsize = 14)
plt.ylabel("Normalized intensity, a. u.", fontsize = 14)
plt.title("Fig. 4: Fit of Raman spectrum ranging from 900 cm$^{-1}$  to 1050 cm$^{-1}$ ",fontsize = 14,fontweight = "bold")
print("rchi-2 = \n"+str(rchi2))